In [3]:
import seaborn as sns
import pandas as pd 
import numpy as np
from scipy.spatial.distance import squareform, pdist
import matplotlib.pyplot as plt
import torch
print("Checkpoint 1")
import anndata as an
import scanpy as sc
import umap
import gc
from importlib import reload
print("Checkpoint 2")

from datasets import Dataset, load_from_disk
from datasets import load_dataset
from geneformer import EmbExtractor
import geneformer

print("Checkpoint 3")

# local imports
import geneformer_utils as gtu

sns.set_style('white')
print("All done!")

Checkpoint 1


/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Checkpoint 2
Checkpoint 3
All done!


In [6]:

"""Load the model"""
#model_path = "/nfs/turbo/umms-indikar/shared/projects/geneformer/fine_tuned_models/geneformer-6L-30M_CellClassifier_cardiomyopathies_220224/"
model_path = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/"
model = gtu.load_model(model_path)

"""Load the data"""
data_path = "/scratch/indikar_root/indikar1/shared_data/geneformer/datasets/TS_Vasculature.dataset"
df = gtu.load_data_as_dataframe(data_path, num_cells=num_cells, shuffle=True)
df.head()

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Step 1: Create a mask to filter rows containing 'fibroblast'
mask = df['cell_ontology_class'].str.contains('fibroblast', case=False, na=False)

# Step 2: Copy the masked DataFrame, preserving the original indexes
filtered_df = df[mask].copy()

# Step 3: Apply the extract_embedding_in_mem function
filtered_df_with_embeddings = extract_embedding_in_mem(model, filtered_df.copy())

#sanity checkpoint
filtered_df_with_enbeddings.to_csv('/home/oliven/geneformer_dev/notebooks/ts_untitled_1_output/filtered_df_with_embeddings.csv')

In [ ]:
# Step 4: Ensure filtered_df_with_embeddings is indexed in the same order as filtered_df
filtered_df_with_embeddings.reset_index(drop=True, inplace=True)
filtered_df.reset_index(drop=True, inplace=True)

# Step 5: Add the embeddings to the filtered dataframe
filtered_df['embeddings'] = filtered_df_with_embeddings['embeddings'].values

# Step 6: Merge the filtered_df with embeddings back to the original df
df['embeddings'] = np.nan  # Initialize the new column with NaN
df.loc[filtered_df.index, 'embeddings'] = filtered_df['embeddings']

# Step 7: Save the filtered dataframe with embeddings as a separate object
fibroblast_df = filtered_df.copy()

# Step 8: Save the original dataframe with embeddings to a new CSV or any other desired format
df.to_csv('/home/oliven/geneformer_dev/notebooks/ts_untitled_1_output/original_df_with_embeddings.csv', index=True)

In [ ]:
# convert filtered stuff to df
filtered_dataset = Dataset.from_pandas(filtered_df)
filtered_dataset.save_to_disk("/home/oliven/geneformer_dev/notebooks/ts_output")
loaded_dataset = Dataset.load_from_disk("/home/oliven/geneformer_dev/notebooks/ts_output")